In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import optuna

In [2]:
# Load dataset
data = load_breast_cancer()
X, y = data.data, data.target

print(X)

print(y)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

In [3]:
# Preprocessing: Splitting the dataset and scaling features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
def objective(trial):
    # Hyperparameters to be tuned
    hidden_layer_sizes = trial.suggest_categorical(
    'hidden_layer_sizes', [
        (50,), (100,), (150,), 
        (50, 50), (100, 50), (150, 100), 
        (100, 100, 100), (50, 100, 50), (30, 50, 30), 
        (200, 100), (100, 100, 100, 100), 
        (200, 150, 100), (100, 200, 100, 50), 
        (30, 30), (20, 20, 20), (30, 30, 30), (40, 40, 40), 
        (20, 20, 20, 20), (30, 30, 30, 30), 
        (25, 25, 25, 25, 25), (20, 20, 20, 20, 20, 20), 
        (15, 15, 15, 15, 15, 15, 15), (10, 10, 10, 10, 10, 10, 10, 10), 
        (5, 5, 5, 5, 5, 5, 5, 5, 5, 5)
    ]
    )


    activation = trial.suggest_categorical('activation', ['tanh', 'relu', 'logistic'])

    solver = trial.suggest_categorical('solver', ['sgd', 'adam'])

    alpha = trial.suggest_float('alpha', 1e-6, 1e-1, log=True)

    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-6, 1e-1, log=True)

    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64, 128, 256, 512])

    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive', 'scheduled'])

    max_iter = trial.suggest_categorical('max_iter', [100, 200, 400, 800, 1600, 3200])

    
    # Model definition
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                          activation=activation,
                          solver=solver,
                          alpha=alpha,
                          learning_rate_init=learning_rate_init,
                          max_iter=1000,  # Increased to ensure convergence
                          random_state=42)
    
    # Cross-validation with scaled data
    score = cross_val_score(model, X_train_scaled, y_train, n_jobs=-1, cv=5).mean()
    return score

In [5]:
# Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, show_progress_bar=True)

[I 2024-03-09 14:10:57,290] A new study created in memory with name: no-name-a9f0f3dc-8c43-4c4d-8afb-46d7df7db595


  0%|          | 0/200 [00:00<?, ?it/s]

/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (150,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choi

[I 2024-03-09 14:10:58,737] Trial 0 finished with value: 0.9736263736263737 and parameters: {'hidden_layer_sizes': (50,), 'activation': 'logistic', 'solver': 'sgd', 'alpha': 0.00015375827507194706, 'learning_rate_init': 0.020605875648994614, 'batch_size': 512, 'learning_rate': 'constant', 'max_iter': 800}. Best is trial 0 with value: 0.9736263736263737.


/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (150,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choi

...

[I 2024-03-09 14:29:05,236] Trial 197 finished with value: 0.9846153846153847 and parameters: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'alpha': 2.9524698435829876e-05, 'learning_rate_init': 5.7200282765593094e-05, 'batch_size': 256, 'learning_rate': 'scheduled', 'max_iter': 3200}. Best is trial 113 with value: 0.9846153846153847.


/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (150,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choi

[I 2024-03-09 14:29:12,775] Trial 198 finished with value: 0.9846153846153847 and parameters: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'alpha': 2.6555082317155454e-05, 'learning_rate_init': 5.293446593585382e-05, 'batch_size': 256, 'learning_rate': 'scheduled', 'max_iter': 3200}. Best is trial 113 with value: 0.9846153846153847.


/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (150,) which is of type tuple.
  warnings.warn(message)
/home/drifterpt03/projects/DL_IEEE/myenv/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choi

[I 2024-03-09 14:29:20,669] Trial 199 finished with value: 0.9846153846153847 and parameters: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'alpha': 3.782281178712897e-05, 'learning_rate_init': 5.1777463765676946e-05, 'batch_size': 256, 'learning_rate': 'scheduled', 'max_iter': 3200}. Best is trial 113 with value: 0.9846153846153847.


In [6]:
# Results
print(f'Best parameters overall: {study.best_params}')
print(f'Best cross-validation score overall: {study.best_value}\n')

# After optimization, retrieve the best 5 trials
best_trials = sorted(study.trials, key=lambda trial: trial.value, reverse=True)[:5]

# Print detailed information for the top 5 trials
print("Top 5 Trials:")
for i, trial in enumerate(best_trials, 1):
    print(f"Rank: {i}")
    print(f"  Trial Number: {trial.number}")
    print(f"  Parameters: {trial.params}")
    print(f"  Cross-validation Score: {trial.value}\n")


Best parameters overall: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'alpha': 3.343268626037649e-05, 'learning_rate_init': 5.350829030791267e-05, 'batch_size': 256, 'learning_rate': 'invscaling', 'max_iter': 3200}
Best cross-validation score overall: 0.9846153846153847

Top 5 Trials:
Rank: 1
  Trial Number: 113
  Parameters: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'alpha': 3.343268626037649e-05, 'learning_rate_init': 5.350829030791267e-05, 'batch_size': 256, 'learning_rate': 'invscaling', 'max_iter': 3200}
  Cross-validation Score: 0.9846153846153847

Rank: 2
  Trial Number: 130
  Parameters: {'hidden_layer_sizes': (200, 100), 'activation': 'relu', 'solver': 'adam', 'alpha': 1.1296491862027729e-05, 'learning_rate_init': 5.099763311947365e-05, 'batch_size': 256, 'learning_rate': 'constant', 'max_iter': 3200}
  Cross-validation Score: 0.9846153846153847

Rank: 3
  Trial Number: 131
  Parameters: {'hidden_layer_sizes': (200

In [7]:
# Use the best parameters found by Optuna, which includes 'max_iter'
best_model = MLPClassifier(**study.best_params).fit(X_train_scaled, y_train)
test_score = best_model.score(X_test_scaled, y_test)
print('Test set score:', test_score)

Test set score: 0.9824561403508771
